## Restaurant Finder Chatbot


### This notebook demonstrates how to use the Gemini API to find restaurants based on location and cuisine.


In [1]:
!pip3 install --upgrade --user google-cloud-aiplatform

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Setup Google Cloud project information and initialize Vertex AI SDK


### follow the link to to get your own project details by following the instructions in it. (https://www.cloudskillsboost.google/paths/236/course_templates/978/labs/461319 to get lab access)

In [3]:
PROJECT_ID = "qwiklabs-gcp-00-54d74cc52666"  # @param {type:"string"}
LOCATION = "us-east4"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

### Using Function Calling in a Chat Session to Answer Users' Questions About Restaurants of a Specific Cuisine in a Specific Location

#### This is how the framework works: Define functions --> wrap functions in a tool --> call Gemini with a tools argument


In [5]:
# Define function to check if there are any restaurants based on Cuisine in a specific location
check_restaurants = FunctionDeclaration(
    name="check_restaurants",
    description="Check if there are restaurants based on Cuisine in a specific location",
    parameters={
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "Location to search for restaurants"},
            "cuisine": {"type": "string", "description": "Cuisine type", "default": "Indian"}
        },
        "required": ["location"]
    },
)

# Define function to find the best restaurant of user requested cuisine based on rating and number of reviews
get_best_restaurant = FunctionDeclaration(
    name="get_best_restaurant",
    description="Get the best restaurant of user requested cuisine based on rating and number of reviews",
    parameters={
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "Location to search for the best restaurant"},
            "cuisine": {"type": "string", "description": "Cuisine type", "default": "Indian"},
            "min_reviews": {"type": "integer", "description": "Minimum number of reviews", "default": 100}
        },
        "required": ["location","cuisine"]
    },
)

In [6]:
resta_tool = Tool(
    function_declarations=[
        check_restaurants,
        get_best_restaurant
    ],
)

### Initialize the Gemini model with Function Calling

In [7]:
model = GenerativeModel(
    "gemini-1.5-pro-preview-0409",
    generation_config=GenerationConfig(temperature=0),
    tools=[resta_tool],
)
chat = model.start_chat()

### Using Yelp API to get the restaurants data

In [8]:
# function to get the content via API
def getcontent_via_api(location,cuisine):
    url = 'https://api.yelp.com/v3/businesses/search'
    api_key = "YOUR_YELP_API_KEY"

    headers = {
        "accept": "application/json",
        "Authorization": "Bearer " + api_key,
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    parameters = {
            'location': location,
            'term': cuisine,
            'limit': 40
        }
    data = requests.get(url, headers=headers, params=parameters).json()
    return data

## Chat Session starts

### User Message 

In [9]:
prompt = """
Are there Indian restaurants in Detroit, Michigan, USA?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "check_restaurants"
  args {
    fields {
      key: "location"
      value {
        string_value: "Detroit, Michigan, USA"
      }
    }
    fields {
      key: "cuisine"
      value {
        string_value: "Indian"
      }
    }
  }
}

### LLM Response 

In [10]:
# Extract the values
response_part = response.candidates[0].content.parts[0].function_call.args
location = response_part["location"]
cuisine = response_part["cuisine"]


response = chat.send_message(
    Part.from_function_response(
        name="Check_restaurants",
        response={
            "content": getcontent_via_api(location,cuisine),
        },
    ),
)
response.candidates[0].content.parts[0]

text: "Yes, there are many Indian restaurants in Detroit, Michigan.\n"

### User Message 

In [11]:
prompt = """
Which is the best Indian restaurant in Detroit, Michigan, USA?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_best_restaurant"
  args {
    fields {
      key: "location"
      value {
        string_value: "Detroit, Michigan, USA"
      }
    }
    fields {
      key: "cuisine"
      value {
        string_value: "Indian"
      }
    }
  }
}

### LLM Response

In [12]:
# Extract the values
response_part = response.candidates[0].content.parts[0].function_call.args
location = response_part["location"]
cuisine = response_part["cuisine"]


response = chat.send_message(
    Part.from_function_response(
        name="check_restaurants",
        response={
            "content": getcontent_via_api(location,cuisine),
        },
    ),
)
response.candidates[0].content.parts[0]

text: "The Himalayan Flames in Dearborn has a 4.6 rating with 254 reviews. It\'s located at 22266 Michigan Ave. \n"

### User Message and LLM instant Response because it can only deal with Restaurants

In [13]:
prompt = """
Can you book an appointment in saloon?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

text: "I am sorry, I cannot fulfill this request. The available tools lack the desired functionality. \n"

### User Message 

In [14]:
prompt = """
Are there Mexican restaurants in Bowling Green, Ohio?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "check_restaurants"
  args {
    fields {
      key: "location"
      value {
        string_value: "Bowling Green, Ohio"
      }
    }
    fields {
      key: "cuisine"
      value {
        string_value: "Mexican"
      }
    }
  }
}

### LLM Response

In [16]:
# Extract the values
response_part = response.candidates[0].content.parts[0].function_call.args
location = response_part["location"]
cuisine = response_part["cuisine"]


response = chat.send_message(
    Part.from_function_response(
        name="Check_restaurants",
        response={
            "content": getcontent_via_api(location,cuisine),
        },
    ),
)
response.candidates[0].content.parts[0]

text: "Yes, there are a few Mexican restaurants in Bowling Green, Ohio. \n"

### User Message 

In [17]:
prompt = """
what are the best Mexican restaurants in Bowling Green, Ohio?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_best_restaurant"
  args {
    fields {
      key: "location"
      value {
        string_value: "Bowling Green, Ohio"
      }
    }
    fields {
      key: "cuisine"
      value {
        string_value: "Mexican"
      }
    }
  }
}

### LLM Response

In [18]:
# Extract the values
response_part = response.candidates[0].content.parts[0].function_call.args
location = response_part["location"]
cuisine = response_part["cuisine"]

response = chat.send_message(
    Part.from_function_response(
        name="Check_restaurants",
        response={
            "content": getcontent_via_api(location,cuisine),
        },
    ),
)
response.candidates[0].content.parts[0]

text: "Guajillo\'s Cocina Mexicana is the best rated, with 4.4 stars and 56 reviews. You can find it at 176 E Wooster St. \n"

### User Message 

In [19]:
prompt = """
are there Indian restaurants in Bowling Green, Ohio?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "check_restaurants"
  args {
    fields {
      key: "location"
      value {
        string_value: "Bowling Green, Ohio"
      }
    }
    fields {
      key: "cuisine"
      value {
        string_value: "Indian"
      }
    }
  }
}

### LLM Response

In [20]:
# Extract the values
response_part = response.candidates[0].content.parts[0].function_call.args
location = response_part["location"]
cuisine = response_part["cuisine"]

response = chat.send_message(
    Part.from_function_response(
        name="Check_restaurants",
        response={
            "content": getcontent_via_api(location,cuisine),
        },
    ),
)
response.candidates[0].content.parts[0]

text: "There are no Indian restaurants in Bowling Green, Ohio. The closest options would be in Toledo, Ohio. \n"